## Select a GPU

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' 

0


## Import models

In [ ]:
import argparse
import sys
import time

from openea.modules.args.args_hander import check_args, load_args
from openea.modules.load.kgs import read_kgs_from_folder
from openea.models.trans import TransD
from openea.models.trans import TransE
from openea.models.trans import TransH
from openea.models.trans import TransR
from openea.models.semantic import DistMult
from openea.models.semantic import HolE
from openea.models.semantic import SimplE
from openea.models.semantic import RotatE
from openea.models.neural import ConvE
from openea.models.neural import ProjE
from openea.approaches import AlignE
from openea.approaches import BootEA
from openea.approaches import JAPE
from openea.approaches import Attr2Vec
from openea.approaches import MTransE
from openea.approaches import IPTransE
from openea.approaches import GCN_Align
from openea.approaches import AttrE
from openea.approaches import IMUSE
from openea.approaches import SEA
from openea.approaches import MultiKE
from openea.approaches import RSN4EA
from openea.approaches import GMNN
from openea.approaches import KDCoE
from openea.approaches import RDGCN
from openea.approaches import BootEA_RotatE
from openea.approaches import BootEA_TransH
from openea.models.basic_model import BasicModel


class ModelFamily(object):
    BasicModel = BasicModel

    TransE = TransE
    TransD = TransD
    TransH = TransH
    TransR = TransR

    DistMult = DistMult
    HolE = HolE
    SimplE = SimplE
    RotatE = RotatE

    ProjE = ProjE
    ConvE = ConvE

    MTransE = MTransE
    IPTransE = IPTransE
    Attr2Vec = Attr2Vec
    JAPE = JAPE
    AlignE = AlignE
    BootEA = BootEA
    GCN_Align = GCN_Align
    GMNN = GMNN
    KDCoE = KDCoE

    AttrE = AttrE
    IMUSE = IMUSE
    SEA = SEA
    MultiKE = MultiKE
    RSN4EA = RSN4EA
    RDGCN = RDGCN
    BootEA_RotatE = BootEA_RotatE
    BootEA_TransH = BootEA_TransH


def get_model(model_name):
    return getattr(ModelFamily, model_name)

## Read args and data

In [3]:
t = time.time()
args = load_args('args/sea_args_15K.json')
args.training_data = args.training_data + 'EN_DE_15K_V1' + '/'
args.dataset_division = '721_5fold/1/'
# args.neoea = True


print(args.embedding_module)
print(args)
remove_unlinked = False
if args.embedding_module == "RSN4EA":
    remove_unlinked = True
kgs = read_kgs_from_folder(args.training_data, args.dataset_division, args.alignment_module, args.ordered,
                           remove_unlinked=remove_unlinked)

load arguments: {'training_data': '../../datasets/', 'output': '../../output/results/', 'dataset_division': '721_5fold', 'embedding_module': 'SEA', 'alignment_module': 'mapping', 'search_module': 'greedy', 'dim': 100, 'init': 'normal', 'ent_l2_norm': True, 'rel_l2_norm': True, 'loss_norm': 'L2', 'margin': 1.5, 'loss': 'margin-based', 'alpha_1': 2.5, 'alpha_2': 0.25, 'neg_sampling': 'uniform', 'neg_triple_num': 1, 'learning_rate': 0.01, 'optimizer': 'Adam', 'max_epoch': 2000, 'batch_size': 5000, 'batch_threads_num': 2, 'test_threads_num': 4, 'ordered': True, 'start_valid': 300, 'eval_freq': 20, 'stop_metric': 'hits1', 'eval_metric': 'inner', 'csls': 10, 'top_k': [1, 5, 10, 50], 'is_save': False, 'eval_norm': True, 'neoea': True, 'neo_sample_size': 512, 'neo_condition_method': 'multiply', 'neo_param': 0.1, 'neo_steps': 1, 'neo_mode': 'full'}
SEA
read relation triples: ../../datasets/EN_DE_15K_V1/rel_triples_1
read relation triples: ../../datasets/EN_DE_15K_V1/rel_triples_2
read attribute

## Init the selected model

In [4]:
model = get_model(args.embedding_module)()
model.set_args(args)
model.set_kgs(kgs)
model.init()

../../output/results/ ../../datasets/EN_DE_15K_V1/ ['..', '..', 'datasets', 'EN_DE_15K_V1'] 721_5fold/1/ SEA
results output folder: ../../output/results/SEA/EN_DE_15K_V1/721_5fold/1/20200922025601/
init neural axioms:
[<openea.approaches.neural_ontology.BasicNeuralAxiom object at 0x7fa9fac9c128>, <openea.approaches.neural_ontology.ECRNeuralAxiom object at 0x7fa9fac9c3c8>, <openea.approaches.neural_ontology.TCRNeuralAxiom object at 0x7fa9fac9c4e0>]
init neural axioms:
[<openea.approaches.neural_ontology.BasicNeuralAxiom object at 0x7fa9fac9c5c0>, <openea.approaches.neural_ontology.ECRNeuralAxiom object at 0x7fa9fac3be80>, <openea.approaches.neural_ontology.TCRNeuralAxiom object at 0x7fa9fac9c7f0>]


## Run & Test

In [6]:
model.run()
model.test()
print("Total run time = {:.3f} s.".format(time.time() - t))

epoch 1, avg. triple loss: 1.2511, cost time: 2.1661s
epoch 1, avg. mapping loss: 6.8124, cost time: 0.3300s
epoch 2, avg. triple loss: 0.7220, cost time: 1.2646s
epoch 2, avg. mapping loss: 6.4916, cost time: 0.1622s
epoch 3, avg. triple loss: 0.5847, cost time: 1.2394s
epoch 3, avg. mapping loss: 6.3179, cost time: 0.1624s
epoch 4, avg. triple loss: 0.5081, cost time: 1.2878s
epoch 4, avg. mapping loss: 6.2190, cost time: 0.1454s
epoch 5, avg. triple loss: 0.4325, cost time: 1.2117s
epoch 5, avg. mapping loss: 6.1639, cost time: 0.1562s
epoch 6, avg. triple loss: 0.3669, cost time: 1.2498s
epoch 6, avg. mapping loss: 6.1352, cost time: 0.1560s
epoch 7, avg. triple loss: 0.3073, cost time: 1.2352s
epoch 7, avg. mapping loss: 6.1168, cost time: 0.1637s
epoch 8, avg. triple loss: 0.2521, cost time: 1.2435s
epoch 8, avg. mapping loss: 6.1055, cost time: 0.1538s
epoch 9, avg. triple loss: 0.2145, cost time: 1.2527s
epoch 9, avg. mapping loss: 6.0962, cost time: 0.1592s
epoch 10, avg. trip

epoch 75, avg. triple loss: 0.0505, cost time: 1.2164s
epoch 75, avg. mapping loss: 6.0627, cost time: 0.1605s
epoch 76, avg. triple loss: 0.0502, cost time: 1.1975s
epoch 76, avg. mapping loss: 6.0627, cost time: 0.1532s
epoch 77, avg. triple loss: 0.0500, cost time: 1.2492s
epoch 77, avg. mapping loss: 6.0624, cost time: 0.1548s
epoch 78, avg. triple loss: 0.0496, cost time: 1.2207s
epoch 78, avg. mapping loss: 6.0624, cost time: 0.1560s
epoch 79, avg. triple loss: 0.0494, cost time: 1.2306s
epoch 79, avg. mapping loss: 6.0625, cost time: 0.1570s
epoch 80, avg. triple loss: 0.0493, cost time: 1.2123s
epoch 80, avg. mapping loss: 6.0625, cost time: 0.1547s
epoch 81, avg. triple loss: 0.0500, cost time: 1.2227s
epoch 81, avg. mapping loss: 6.0625, cost time: 0.1578s
epoch 82, avg. triple loss: 0.0490, cost time: 1.2548s
epoch 82, avg. mapping loss: 6.0625, cost time: 0.1599s
epoch 83, avg. triple loss: 0.0486, cost time: 1.2364s
epoch 83, avg. mapping loss: 6.0625, cost time: 0.1572s
e

epoch 148, avg. triple loss: 0.0434, cost time: 1.2719s
epoch 148, avg. mapping loss: 6.0609, cost time: 0.1636s
epoch 149, avg. triple loss: 0.0432, cost time: 1.2325s
epoch 149, avg. mapping loss: 6.0608, cost time: 0.1560s
epoch 150, avg. triple loss: 0.0432, cost time: 1.2357s
epoch 150, avg. mapping loss: 6.0608, cost time: 0.1532s
epoch 151, avg. triple loss: 0.0431, cost time: 1.2421s
epoch 151, avg. mapping loss: 6.0608, cost time: 0.1618s
epoch 152, avg. triple loss: 0.0428, cost time: 1.2596s
epoch 152, avg. mapping loss: 6.0609, cost time: 0.1590s
epoch 153, avg. triple loss: 0.0435, cost time: 1.2277s
epoch 153, avg. mapping loss: 6.0607, cost time: 0.1551s
epoch 154, avg. triple loss: 0.0424, cost time: 1.2003s
epoch 154, avg. mapping loss: 6.0609, cost time: 0.1493s
epoch 155, avg. triple loss: 0.0431, cost time: 1.2330s
epoch 155, avg. mapping loss: 6.0607, cost time: 0.1555s
epoch 156, avg. triple loss: 0.0435, cost time: 1.2349s
epoch 156, avg. mapping loss: 6.0608, co

epoch 221, avg. triple loss: 0.0411, cost time: 1.2257s
epoch 221, avg. mapping loss: 6.0602, cost time: 0.1521s
epoch 222, avg. triple loss: 0.0400, cost time: 1.2367s
epoch 222, avg. mapping loss: 6.0602, cost time: 0.1625s
epoch 223, avg. triple loss: 0.0406, cost time: 1.2294s
epoch 223, avg. mapping loss: 6.0602, cost time: 0.1622s
epoch 224, avg. triple loss: 0.0400, cost time: 1.2360s
epoch 224, avg. mapping loss: 6.0601, cost time: 0.1611s
epoch 225, avg. triple loss: 0.0389, cost time: 1.2758s
epoch 225, avg. mapping loss: 6.0601, cost time: 0.1575s
epoch 226, avg. triple loss: 0.0401, cost time: 1.2615s
epoch 226, avg. mapping loss: 6.0600, cost time: 0.1598s
epoch 227, avg. triple loss: 0.0405, cost time: 1.2365s
epoch 227, avg. mapping loss: 6.0600, cost time: 0.1607s
epoch 228, avg. triple loss: 0.0404, cost time: 1.2430s
epoch 228, avg. mapping loss: 6.0601, cost time: 0.1606s
epoch 229, avg. triple loss: 0.0410, cost time: 1.2711s
epoch 229, avg. mapping loss: 6.0601, co

epoch 294, avg. triple loss: 0.0382, cost time: 1.2355s
epoch 294, avg. mapping loss: 6.0595, cost time: 0.1561s
epoch 295, avg. triple loss: 0.0386, cost time: 1.2112s
epoch 295, avg. mapping loss: 6.0594, cost time: 0.1561s
epoch 296, avg. triple loss: 0.0381, cost time: 1.2275s
epoch 296, avg. mapping loss: 6.0595, cost time: 0.1560s
epoch 297, avg. triple loss: 0.0386, cost time: 1.2221s
epoch 297, avg. mapping loss: 6.0595, cost time: 0.1551s
epoch 298, avg. triple loss: 0.0378, cost time: 1.2527s
epoch 298, avg. mapping loss: 6.0596, cost time: 0.1588s
epoch 299, avg. triple loss: 0.0383, cost time: 1.2326s
epoch 299, avg. mapping loss: 6.0595, cost time: 0.1556s
epoch 300, avg. triple loss: 0.0384, cost time: 1.2211s
epoch 300, avg. mapping loss: 6.0594, cost time: 0.1554s
quick results: hits@[1, 5, 10, 50] = [52.733 69.867 77.267 90.333]%, time = 0.644 s 
epoch 301, avg. triple loss: 0.0386, cost time: 1.2308s
epoch 301, avg. mapping loss: 6.0595, cost time: 0.1662s
epoch 302, 

epoch 364, avg. triple loss: 0.0366, cost time: 1.2811s
epoch 364, avg. mapping loss: 6.0591, cost time: 0.1667s
epoch 365, avg. triple loss: 0.0365, cost time: 1.2282s
epoch 365, avg. mapping loss: 6.0591, cost time: 0.1603s
epoch 366, avg. triple loss: 0.0373, cost time: 1.2270s
epoch 366, avg. mapping loss: 6.0592, cost time: 0.1627s
epoch 367, avg. triple loss: 0.0365, cost time: 1.3000s
epoch 367, avg. mapping loss: 6.0590, cost time: 0.1628s
epoch 368, avg. triple loss: 0.0368, cost time: 1.2434s
epoch 368, avg. mapping loss: 6.0591, cost time: 0.1615s
epoch 369, avg. triple loss: 0.0374, cost time: 1.2404s
epoch 369, avg. mapping loss: 6.0591, cost time: 0.1626s
epoch 370, avg. triple loss: 0.0378, cost time: 1.2523s
epoch 370, avg. mapping loss: 6.0591, cost time: 0.1568s
epoch 371, avg. triple loss: 0.0376, cost time: 1.2402s
epoch 371, avg. mapping loss: 6.0591, cost time: 0.1587s
epoch 372, avg. triple loss: 0.0378, cost time: 1.2460s
epoch 372, avg. mapping loss: 6.0591, co

epoch 435, avg. triple loss: 0.0366, cost time: 1.2277s
epoch 435, avg. mapping loss: 6.0587, cost time: 0.1608s
epoch 436, avg. triple loss: 0.0367, cost time: 1.2367s
epoch 436, avg. mapping loss: 6.0588, cost time: 0.1641s
epoch 437, avg. triple loss: 0.0362, cost time: 1.2434s
epoch 437, avg. mapping loss: 6.0587, cost time: 0.1644s
epoch 438, avg. triple loss: 0.0369, cost time: 1.2493s
epoch 438, avg. mapping loss: 6.0587, cost time: 0.1648s
epoch 439, avg. triple loss: 0.0358, cost time: 1.2449s
epoch 439, avg. mapping loss: 6.0587, cost time: 0.1546s
epoch 440, avg. triple loss: 0.0359, cost time: 1.2109s
epoch 440, avg. mapping loss: 6.0587, cost time: 0.1590s
quick results: hits@[1, 5, 10, 50] = [55.667 72.533 80.4   91.667]%, time = 0.653 s 
epoch 441, avg. triple loss: 0.0361, cost time: 1.2197s
epoch 441, avg. mapping loss: 6.0587, cost time: 0.1582s
epoch 442, avg. triple loss: 0.0359, cost time: 1.2292s
epoch 442, avg. mapping loss: 6.0587, cost time: 0.1519s
epoch 443, 

epoch 505, avg. triple loss: 0.0352, cost time: 1.2416s
epoch 505, avg. mapping loss: 6.0584, cost time: 0.1637s
epoch 506, avg. triple loss: 0.0347, cost time: 1.2247s
epoch 506, avg. mapping loss: 6.0585, cost time: 0.1523s
epoch 507, avg. triple loss: 0.0348, cost time: 1.2316s
epoch 507, avg. mapping loss: 6.0585, cost time: 0.1601s
epoch 508, avg. triple loss: 0.0356, cost time: 1.2518s
epoch 508, avg. mapping loss: 6.0584, cost time: 0.1576s
epoch 509, avg. triple loss: 0.0353, cost time: 1.2345s
epoch 509, avg. mapping loss: 6.0585, cost time: 0.1634s
epoch 510, avg. triple loss: 0.0353, cost time: 1.2519s
epoch 510, avg. mapping loss: 6.0585, cost time: 0.1581s
epoch 511, avg. triple loss: 0.0350, cost time: 1.2881s
epoch 511, avg. mapping loss: 6.0584, cost time: 0.1604s
epoch 512, avg. triple loss: 0.0350, cost time: 1.2599s
epoch 512, avg. mapping loss: 6.0584, cost time: 0.1618s
epoch 513, avg. triple loss: 0.0351, cost time: 1.2748s
epoch 513, avg. mapping loss: 6.0584, co

epoch 576, avg. triple loss: 0.0354, cost time: 1.2184s
epoch 576, avg. mapping loss: 6.0582, cost time: 0.1589s
epoch 577, avg. triple loss: 0.0343, cost time: 1.2209s
epoch 577, avg. mapping loss: 6.0582, cost time: 0.1586s
epoch 578, avg. triple loss: 0.0348, cost time: 1.2355s
epoch 578, avg. mapping loss: 6.0582, cost time: 0.1655s
epoch 579, avg. triple loss: 0.0349, cost time: 1.2465s
epoch 579, avg. mapping loss: 6.0582, cost time: 0.1599s
epoch 580, avg. triple loss: 0.0348, cost time: 1.2373s
epoch 580, avg. mapping loss: 6.0583, cost time: 0.1531s
quick results: hits@[1, 5, 10, 50] = [57.533 74.133 81.133 92.   ]%, time = 0.633 s 
epoch 581, avg. triple loss: 0.0341, cost time: 1.2378s
epoch 581, avg. mapping loss: 6.0582, cost time: 0.1616s
epoch 582, avg. triple loss: 0.0351, cost time: 1.2135s
epoch 582, avg. mapping loss: 6.0582, cost time: 0.1543s
epoch 583, avg. triple loss: 0.0352, cost time: 1.2548s
epoch 583, avg. mapping loss: 6.0583, cost time: 0.1629s
epoch 584, 

epoch 645, avg. mapping loss: 6.0581, cost time: 0.2612s
epoch 646, avg. triple loss: 0.0345, cost time: 2.7713s
epoch 646, avg. mapping loss: 6.0581, cost time: 0.2655s
epoch 647, avg. triple loss: 0.0342, cost time: 2.6027s
epoch 647, avg. mapping loss: 6.0581, cost time: 0.2485s
epoch 648, avg. triple loss: 0.0344, cost time: 2.5676s
epoch 648, avg. mapping loss: 6.0581, cost time: 0.2430s
epoch 649, avg. triple loss: 0.0342, cost time: 2.6934s
epoch 649, avg. mapping loss: 6.0581, cost time: 0.2654s
epoch 650, avg. triple loss: 0.0339, cost time: 2.7873s
epoch 650, avg. mapping loss: 6.0581, cost time: 0.3273s
epoch 651, avg. triple loss: 0.0343, cost time: 2.7479s
epoch 651, avg. mapping loss: 6.0580, cost time: 0.2487s
epoch 652, avg. triple loss: 0.0343, cost time: 2.7785s
epoch 652, avg. mapping loss: 6.0581, cost time: 0.3165s
epoch 653, avg. triple loss: 0.0346, cost time: 2.3175s
epoch 653, avg. mapping loss: 6.0581, cost time: 0.2701s
epoch 654, avg. triple loss: 0.0348, co